<a href="https://colab.research.google.com/github/amirafshari/projects/blob/main/HubermanLabPodcast_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-gpu
!pip install gpt4all

# Load

In [2]:
with open('/content/EP002 - How Your Brain Works & Changes.txt', 'r', encoding='utf-8-sig') as f:
    data = f.read()

In [3]:
data

"Welcome to the Uberman Lab podcast where we discuss science and science-based tools for everyday life. I'm Andrew Uberman and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. For today's podcast, we're going to talk about the parts list of the nervous system. Now that might sound boring, but these are the bits and pieces that together make up everything about your experience of life. From what you think about to what you feel, what you imagine, and what you accomplish from the day you're born until the day you die. That parts list is really incredible because it has a history associated with it that really provides a window into all sorts of things like engineering, warfare, religion, and philosophy. So I'm going to share with you the parts list that makes up who you are through the lens of some of those other aspects of life and other aspects of the history of the discovery of the nervous system. By the end of this podcast, I promise you're going to u

# Split

In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [5]:
text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    separator='.',
)

In [6]:
doc = text_splitter.create_documents([data])

In [7]:
splits = text_splitter.split_documents(doc)

In [8]:
splits

[Document(page_content="Welcome to the Uberman Lab podcast where we discuss science and science-based tools for everyday life. I'm Andrew Uberman and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. For today's podcast, we're going to talk about the parts list of the nervous system. Now that might sound boring, but these are the bits and pieces that together make up everything about your experience of life"),
 Document(page_content="Now that might sound boring, but these are the bits and pieces that together make up everything about your experience of life. From what you think about to what you feel, what you imagine, and what you accomplish from the day you're born until the day you die. That parts list is really incredible because it has a history associated with it that really provides a window into all sorts of things like engineering, warfare, religion, and philosophy"),
 Document(page_content="That parts list is really incredible because it has a 

# Embed

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
hf = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Store

In [11]:
import faiss
from langchain.vectorstores.faiss import FAISS

In [12]:
texts = []
for txt in doc:
  texts.append(txt.page_content)

In [13]:
vectorstore = FAISS.from_texts(texts, hf)
faiss.write_index(vectorstore.index, "index.index")

In [18]:
question = "Who is andrew huberman?"
answ = vectorstore.similarity_search(question)
answ

[Document(page_content="Welcome to the Uberman Lab podcast where we discuss science and science-based tools for everyday life. I'm Andrew Uberman and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. For today's podcast, we're going to talk about the parts list of the nervous system. Now that might sound boring, but these are the bits and pieces that together make up everything about your experience of life"),
 Document(page_content='Sherrington want to know about prize for his work in mapping some of the circuitry the connections between nerve cells that give rise to movement and he said movement is the final common pathway the other way to think about it is that one of the reasons that our central nervous system our brain and spinal cord include this stuff in our skull but also connect so heavily to the body is because most everything that we experience including our thoughts and feelings was really designed to either impact our behavior or not'),
 Doc

# LLMs

In [ ]:
# from transformers import pipeline, set_seed
# generator = pipeline('text-generation', model='gpt2')
# generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

In [ ]:
# !wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin

In [15]:
from langchain.llms import GPT4All
llm = GPT4All(model="/content/llama-2-7b-chat.ggmlv3.q4_0.bin")

Found model file at  /content/llama-2-7b-chat.ggmlv3.q4_0.bin


In [19]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': 'Who is andrew huberman?',
 'result': ' Andrew Uberman is a professor of neurobiology and ophthalmology at Stanford School of Medicine.'}